<img src="Archivos/miad4.png" width=800x>

## Interactividad usando `panel`

En este taller podrás poner en práctica los conceptos aprendidos acerca del uso del paquete `panel`. Usarás `panel` con el fin de crear dashboards como herramienta de apoyo al momento de resolver problemas específicos en un contexto de Analytics.

Instrucciones y condiciones generales: Esta es una actividad calificada y, por lo tanto, debe ser resuelta individualmente.

### Al desarrollar este taller podrás verificar tu progreso en:

1. Programar la funcionalidad de un dashboard en `panel`.
2. Agregar gráficos y widgets para hacer visualizaciones agradables en `panel`.
3. Personalizar dashobards para generar distintas visualizaciones interactivas en `panel`.

In [1]:
import panel as pn
import plotly.graph_objects as go
import pandas as pd
pn.extension('plotly')

## Pregunta 1

Con la base de datos *COVID-19.csv* crea un dashboard en `panel` que:

1. Permita crear un diagrama de barras horizontal que muestre los casos de COVID-19 de acuerdo a la selección de los siguentes criterios:<br />
    a. Casos o muertes.<br />
    b. Periodo: Histórico, Últimos 7 días y Últimas 24 horas.<br />
    Pista: utilice un control de tipo `RadioButtonGroup` para cada uno de los dos criterios.<br />
2. Contenga un diagrama de barras horizontal de las muertes por millón de personas.
3. Permita visualizar a tráves de un diagrama de barras horizontal los casos en cada una de las regiones de la Organización Munidal de la Salud.

Nota: a excpeción de las regiones, muestre los 10 mayores valores en cada gráfico.

| <center>Variable</center> |<center>Descripción</center> | 
| :-- | :-- | 
| Name | País |
| WHO Region | Región de la OMS |
| Cases - cumulative total | Total de casos hasta el momento |
| Cases - cumulative total per 1 million population | Número de casos acumulado por millón de personas |
| Cases - newly reported in last 7 days | Número de casos nuevos en los últimos 7 días |
| Cases - newly reported in last 24 hours | Número de casos nuevos en las últimas 24 horas |
| Deaths - cumulative total | Total de muertes hasta el momento |
| Deaths - cumulative total per 1 million population | Número de muertes acumulado por millón de personas |
| Deaths - newly reported in last 7 days | Número de muertes nuevas en los últimos 7 días |
| Deaths - newly reported in last 24 hours | Número de muertes nuevas en las últimas 24 horas |

In [2]:
# YOUR CODE HERE
datos = pd.read_csv('Archivos/COVID-19.csv')
datos = pd.DataFrame(datos)
datos.head()


,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 1 million population,Cases - newly reported in last 7 days,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 1 million population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 24 hours,Transmission Classification
0,United States of America,Americas,26547977,80204.73,871365,129961,455735,1376.83,22562,3549,Community transmission
1,India,South-East Asia,10826363,7845.17,80180,12059,154996,112.32,722,78,Clusters of cases
2,Brazil,Americas,9447165,44444.82,328652,50872,230034,1082.21,7368,1239,Community transmission
3,Russian Federation,Europe,3967281,27185.36,116842,16048,76661,525.31,3479,432,Clusters of cases
4,The United Kingdom,Europe,3929839,57888.79,133747,18262,112092,1651.18,6521,828,Community transmission


In [3]:
# YOUR CODE HERE
titulo = '# Casos de COVID'

In [4]:
# YOUR CODE HERE
seleccion1 = pn.widgets.RadioButtonGroup(name='seleccion1', options= ["Casos","Muertes"])
seleccion2 = pn.widgets.RadioButtonGroup(name='seleccion2', options= ["Historico","Ultimos 7 dias", "Ultimas 24 Horas"])


In [5]:
# YOUR CODE HERE
@pn.depends(seleccion1, seleccion2)
# Definimos la función que se encarga de la interactividad del tablero de control.
def funcion_interactiva(seleccion1,seleccion2): 
    
    df=datos
    if seleccion1== "Casos":
        column='Cases'
    if seleccion1=="Muertes":
        column='Deaths'

    
    if seleccion2=="Ultimos 7 dias":
        column_periodo=column+' - newly reported in last 7 days'
    if seleccion2=="Ultimas 24 Horas":
        column_periodo=column+' - newly reported in last 24 hours'
    if seleccion2=="Historico":
        column_periodo=column+' - cumulative total'
        
    print(column_periodo)
        
    df=df.groupby('Name').sum().reset_index()
    df=df[['Name',column_periodo]]
    
    df.sort_values(by=[column_periodo], inplace=True, ascending=False)
    
    df=df.head(10)

    fig = go.Figure(go.Bar(
                            x=df[column_periodo],
                            y=df['Name'],
                            orientation='h'                                     
                          )
                   )
    fig.update_layout(
                       
                        title= str(seleccion1)+' en ' + str(seleccion2)
               
                     )
    
                    
                     
    
    return fig

In [6]:
# YOUR CODE HERE
def muertes_millon():
    df=datos
    
    df=df.groupby('Name').sum().reset_index()
    df=df[['Name','Deaths - cumulative total per 1 million population']]
    df.sort_values(by=['Deaths - cumulative total per 1 million population'], inplace=True, ascending=False)
    
    df=df.head(10)
    fig = go.Figure(go.Bar(
                            x=df['Deaths - cumulative total per 1 million population'],
                            y=df['Name'],
                            orientation='h'                                     
                          )
                   )
    fig.update_layout(
                        title= "Muertes acumulativas por millon"
                              
                     )
    
    return fig


def casos_por_region():
    df=datos
    
    df=df.groupby('WHO Region').sum().reset_index()
    df=df[['WHO Region','Cases - cumulative total']]
    df.sort_values(by=['Cases - cumulative total'], inplace=True, ascending=True)
    
    fig = go.Figure(go.Bar(
                            x=df['Cases - cumulative total'],
                            y=df['WHO Region'],
                            orientation='h'       
                          )
                   )
    fig.update_layout(
  
                        title= "Casos por region",
                        
                     )
    
            
    return fig
    

In [7]:
# YOUR CODE HERE
tablero_control = pn.Row(pn.Column(titulo, seleccion1,seleccion2, muertes_millon, casos_por_region ), funcion_interactiva)

Cases - cumulative total


In [8]:
tablero_control.servable()

Deaths - cumulative total 
Deaths - newly reported in last 24 hours 
Deaths - newly reported in last 7 days

Row
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(name='seleccion1', options=['Casos', 'Muertes'], value='Casos')
        [2] RadioButtonGroup(name='seleccion2', options=['Historico', ...], value='Historico')
        [3] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive00105')
        [4] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive00111')
    [1] ParamFunction(function)

In [9]:
# YOUR CODE HERE
tablero_control.show()

Launching server at http://localhost:56724


In [219]:
# YOUR CODE HERE


## Créditos

**Autor(es)**: Juan David Reyes Jaimes, Diego Alejandro Cely Gomez

**Última actualización:** 26/08/2021